In [ ]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
#from visual import *
import numpy as np
import timeit
from math import *

In [ ]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds, draw_agents(agents, width, height)


def draw_agents(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [66]:
def couzin(agents, speeds, N, s, noise, rr=1, ro=2, ra=3):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir 

def gbias(speeds, N, s, prop, bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * bias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir

def accuracy(speeds, N, bias):
    dev = 0.0
    for i in range(N):
        dev = dev + np.arccos(np.dot(bias,normalized(speeds[i])))
    dev_avg = dev/(N*2*(np.pi))
    print (dev_avg)
    
def elongation(agents, N):
    cn = np.array([0.0, 0.0])
    for i in range(N):
        cn = agents
    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir

def gueron():
    return

In [70]:
def simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        couzin(agents, speeds, N, s, noise,100,1900,2000)
        gbias(speeds, N, s, prop, bias, weight)
        accuracy(speeds, N, bias)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)
        #print(v_avg)
        #v_avg = get_v_avg(speeds)
        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    

#simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias)
simulate(2000, 0.1, 1, 50, 500, 500, 2, 0.02, 0.4,0.5, np.array([0.0,-1.0]))

0.280015760072
0.252968484638
0.235544955225
0.233675859245
0.223308946651
0.21795996666
0.225225312054
0.214016144976
0.198339750502
0.202849774534
0.211816577375
0.196632000292
0.192118922578
0.18488477079
0.18701318739
0.194818858685
0.191552085269
0.191357519902
0.201682774079
0.188361727059
0.182044521514
0.177813707751
0.176085378542
0.174186689246
0.165471905267
0.176557183769
0.176985975029
0.170037032781
0.16653870727
0.159207067279
0.159382191776
0.148056873925
0.140944541721
0.134924262105
0.133093931509
0.126791093862
0.123776429093
0.117499781961
0.130707277328
0.133970320282
0.12380371172
0.118886767773
0.113745620536
0.115171627075
0.107970501669
0.100260694011
0.103761746123
0.105209964183
0.111284263438
0.100794975251
0.0991480135615
0.0974033046443
0.0956933236473
0.0940859909472
0.0967299873019
0.095646914836
0.0961048885415
0.104536799504
0.102308398808
0.107123097322
0.100118132717
0.102234838241
0.0986281284791
0.0966020341271
0.0939850497359
0.101431443357
0.0995

0.000558097432342
0.000546791046863
0.000535708898135
0.000524846821343
0.000502054962678
0.000511698562191
0.00049089470014
0.000501532411557
0.000492611160978
0.000487944065485
0.000475879523285
0.000464803988968
0.000453527827034
0.000462106124822
0.000448795388302
0.000438901146367
0.000426763541253
0.000421198134964
0.000412074981685
0.000404114773626
0.000396285832291
0.000388597041892
0.00038105255071
0.000373656139303
0.000358142511464
0.000362396056383
0.000355533644175
0.00033803622398
0.00033964611463
0.0003329756087
0.000326569063143
0.000320173498166
0.000313882166778
0.000307709404455
0.000301658484385
0.000295729606725
0.000289921676734
0.00028179912978
0.000277454520174
0.000268826183998
0.000261263357914
0.000254488875148
0.000248287909356
0.000242506757916
0.000237039617419
0.00022714534066
0.000222930228556
0.000212511387946
0.000206919936412
0.000201909505608
0.000197243029476
0.000192839072708
0.000188649698444
0.000184647691443
0.000180815352578
0.000177139790312


7.69434837037e-08
7.52195717467e-08
7.36730681521e-08
7.21605394777e-08
7.06748312936e-08
6.92233613224e-08
6.78055736356e-08
6.64192446021e-08
6.50505854294e-08
6.37115977981e-08
6.24121314464e-08
6.11280663209e-08
5.98261185594e-08
5.85999223432e-08
5.7393174649e-08
5.62198211945e-08
5.50590030358e-08
5.39329096736e-08
5.2832583909e-08
5.17399596742e-08
5.0682383836e-08
4.96377736054e-08
4.86279418658e-08
4.76329515556e-08
4.66553978851e-08
4.57075447501e-08
4.4760829745e-08
4.38428275959e-08
4.29469768208e-08
4.20625182207e-08
4.11968192626e-08
4.03562803819e-08
3.95305017615e-08
3.87110812118e-08
3.79190363289e-08
3.71484993336e-08
3.63925194881e-08
3.56382266093e-08
3.40944238773e-08
3.47236938387e-08
3.32903830209e-08
3.4035959645e-08
3.34092651455e-08
3.31039850629e-08
3.2271330822e-08
3.15124737008e-08
3.07317836918e-08
3.13126711132e-08
3.03953417654e-08
2.97470103875e-08
2.89158901927e-08
2.85463803482e-08
2.79390238389e-08
2.73859460334e-08
2.68526769962e-08
2.63230118444e-0